In [19]:
import numpy as np
from patch_generator import smash_n_reconstruct
import filters as f
import cv2
from matplotlib import pyplot as plt
import pandas as pd
import os
import concurrent.futures
from tqdm import tqdm
import random
import gc

### Objective: 
Identify features to extract from rich texture and poor texture images

1. Loading image paths:

In [20]:
path_ai = './test_imgs/AI_SD_ukiyo-e/'
ai_img = os.listdir(path_ai)
ai_img[:2]

['9-250992346-230466.jpg', '9-250992346-255116.jpg']

In [21]:
path_real = './test_imgs/ukiyo_e'
real_img = os.listdir(path_real)
real_img[:2]

['adachi-ginko_114.jpg', 'adachi-ginko_116.jpg']

**Pixel Fluctuation Degree**:

$$
 l_{div} = \sum_{i=1}^{M}\sum_{j=1}^{M-1}(|x_{i,j}-x_{i,j+1}|) + \sum_{i=1}^{M-1}\sum_{j=1}^{M}(|x_{i,j}-x_{i+1,j}|) 
$$
$$
 + \sum_{i=1}^{M-1}\sum_{j=1}^{M-1}(|x_{i,j}-x_{i+1,j+1}|) + \sum_{i=1}^{M-1}\sum_{j=1}^{M-1}(|x_{i,j}-x_{i,j+1}|)
$$

In [22]:
def get_l1(v,x,y):
    l1=0
    # 1 to m, 1 to m-1
    for i in range(0,y-1):
        for j  in range(0,x):
            l1+=abs(v[j][i]-v[j][i+1])
    return l1

def get_l2(v,x,y):
    l2=0
    # 1 to m-1, 1 to m
    for i in range(0,y):
        for j  in range(0,x-1):
            l2+=abs(v[j][i]-v[j+1][i])
    return l2

def get_l3l4(v,x,y):
    l3=l4=0
    # 1 to m-1, 1 to m-1
    for i in range(0,y-1):
        for j  in range(0,x-1):
            l3+=abs(v[j][i]-v[j+1][i+1])
            l4+=abs(v[j+1][i]-v[j][i+1])

    return l3+l4

def pixel_fluctuation(patch:np.array)->int:
    x,y = patch.shape
    l1=l2=l3l4=0
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        future_l1 = executor.submit(get_l1,patch,x,y)
        future_l2 = executor.submit(get_l2,patch,x,y)
        future_l3l4 = executor.submit(get_l3l4,patch,x,y)

        l1 = future_l1.result()
        l2 = future_l2.result()
        l3l4 = future_l3l4.result()

    return  l1+l2+l3l4

In [23]:
y = []
X = []

## Feature:
We get the ratio of:

_pixel_fluctuation_(Rich Texture patch) / _pixel_fluctuation_(Poor Texture patch)

In [24]:
pb = tqdm(total=1500)
for img_name in real_img[:1500]:
    rt,pt = smash_n_reconstruct(input_path=os.path.join(path_real,img_name))
    rt = np.asarray((cv2.cvtColor(f.apply_all_filters(rt),cv2.COLOR_RGB2GRAY))).astype(np.int32)
    pt = np.asarray((cv2.cvtColor(f.apply_all_filters(pt),cv2.COLOR_RGB2GRAY))).astype(np.int32)
    X.append((pixel_fluctuation(pt)/pixel_fluctuation(rt)))
    y.append(0)
    pb.update(1)
pb.close()

100%|██████████| 1500/1500 [12:41<00:00,  1.97it/s]


In [25]:
pb = tqdm(total=1500)
for img_name in ai_img[:1500]:
    rt,pt = smash_n_reconstruct(input_path=os.path.join(path_ai,img_name))
    rt = np.asarray((cv2.cvtColor(f.apply_all_filters(rt),cv2.COLOR_RGB2GRAY))).astype(np.int32)
    pt = np.asarray((cv2.cvtColor(f.apply_all_filters(pt),cv2.COLOR_RGB2GRAY))).astype(np.int32)
    X.append((pixel_fluctuation(pt)/pixel_fluctuation(rt)))
    y.append(1)
    pb.update(1)
pb.close()

100%|██████████| 1500/1500 [13:07<00:00,  1.90it/s]


In [26]:
df = pd.DataFrame()

In [27]:
df['pixel_fluctuation_ratio'] = X
df['is_ai'] = y

In [28]:
rmean = df[df['is_ai']==0]['pixel_fluctuation_ratio'].mean()
df[df['is_ai']==0]['pixel_fluctuation_ratio'].mean(), df[df['is_ai']==0]['pixel_fluctuation_ratio'].median()

(0.7886128462784631, 0.8126382956702631)

In [29]:
aimean = df[df['is_ai']==1]['pixel_fluctuation_ratio'].mean()
df[df['is_ai']==1]['pixel_fluctuation_ratio'].mean(), df[df['is_ai']==1]['pixel_fluctuation_ratio'].median()

(0.9168690037314776, 0.928371899215739)

The variance for this metric is surprisingly low for ai generated images

In [31]:
df[df['is_ai']==0]['pixel_fluctuation_ratio'].var(),df[df['is_ai']==1]['pixel_fluctuation_ratio'].var()

(0.018661744075397894, 0.004523052978360972)

In [32]:
ai_true = 0
for i in tqdm(range(100)):
    img_name = random.choice(ai_img[1499:])
    rt,pt = smash_n_reconstruct(input_path=os.path.join(path_ai,img_name))
    rt = np.asarray((cv2.cvtColor(f.apply_all_filters(rt),cv2.COLOR_RGB2GRAY))).astype(np.int32)
    pt = np.asarray((cv2.cvtColor(f.apply_all_filters(pt),cv2.COLOR_RGB2GRAY))).astype(np.int32)
    pr = pixel_fluctuation(pt)/pixel_fluctuation(pt)
    if(abs(pr-rmean)>abs(pr-aimean)):
        ai_true+=1

print(ai_true/100)

100%|██████████| 100/100 [00:50<00:00,  1.97it/s]

1.0


In [33]:
real_true = 0
for i in tqdm(range(100)):
    img_name = random.choice(real_img[1499:])
    rt,pt = smash_n_reconstruct(input_path=os.path.join(path_real,img_name))
    rt = np.asarray((cv2.cvtColor(f.apply_all_filters(rt),cv2.COLOR_RGB2GRAY))).astype(np.int32)
    pt = np.asarray((cv2.cvtColor(f.apply_all_filters(pt),cv2.COLOR_RGB2GRAY))).astype(np.int32)
    pr = pixel_fluctuation(pt)/pixel_fluctuation(rt)
    if(abs(pr-rmean)<abs(pr-aimean)):
        real_true+=1

print(real_true/100)

100%|██████████| 100/100 [00:50<00:00,  1.99it/s]

0.62


In [34]:
rt,pt = smash_n_reconstruct(input_path='./test_imgs/fake.png')
rt = np.asarray((cv2.cvtColor(f.apply_all_filters(rt),cv2.COLOR_RGB2GRAY))).astype(np.int32)
pt = np.asarray((cv2.cvtColor(f.apply_all_filters(pt),cv2.COLOR_RGB2GRAY))).astype(np.int32)
pixel_fluctuation(pt)/pixel_fluctuation(rt)

0.7253068264621273

In [35]:
len(real_img), len(ai_img)

(5000, 5308)

In [36]:
gc.collect()

9